In [ ]:
%matplotlib inline
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn import preprocessing
from sklearn.datasets import make_regression
from keras.layers import Dense
from keras.layers import Flatten
from keras.models import Sequential
from keras.optimizers import SGD
from dataset_manager import *
from predict import *
from trame_show import *
from datetime import date, datetime
from termcolor import colored
import random

train_dataset, test_dataset = get_dataset()
shift_letter(train_dataset, test_dataset)
d_list = list(test_dataset)

#train_dataset, test_dataset = filtre(train_dataset, test_dataset, 0.3)

X_train, Y_train = get_X_Y_vectorized_int(train_dataset)
X_test, Y_test = get_X_Y_vectorized_int(test_dataset)

X_train, Y_train = shuffle_X_Y(X_train, Y_train)
X_test, Y_test = shuffle_X_Y(X_test, Y_test)

#X_login_mdp, _ = get_pics_from_file(f"../tohack/pics_LOGINMDP.bin")
#X_login_mdp_filtred = filtre_result(X_login_mdp, np.array(train_dataset.get('NOKEY')).mean(0), 0.3)

#for l in range(0, len(X_login_mdp_filtred), 500):
#    trame_showw(X_login_mdp_filtred, "LOGINMDP", l)
#std_scale = preprocessing.StandardScaler().fit(X_train)
#X_train = std_scale.transform(X_train)

In [ ]:
def train_model(epochs, batch_size, couches_neu):
    """Réseau de neurones"""
    delta_time = datetime.now().time()
    model = Sequential()
    model.add(Dense(17))
    for _ in range(6):
        model.add(Dense(128, activation='sigmoid'))
    model.add(Dense(len(d_list)))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

    model.fit(np.array(X_train), np.array(Y_train), epochs=epochs, batch_size=batch_size, )
    
    test_loss, test_acc = model.evaluate(np.array(X_test), np.array(Y_test), verbose=0)

    accuracy = compute_accuracy_keras(model, d_list, X_test, Y_test)
    real_accu = sum(accuracy.values()) / len(d_list)
    
    #delta_time = datetime.combine(date.min, datetime.now().time()) - datetime.combine(date.min, delta_time)
    #plt.scatter([couches_neu], [real_accu], label='sortie_nbr={0}, time={1}s, 1st accu={2}'.format(couches_neu, round(delta_time.seconds, 3), round(test_acc, 3)))
    #print('sortie_nbr={0}, real_accu={1}, 1st accu={2}   {3}'.format(couches_neu, round(real_accu, 3), round(test_acc, 3), test_loss))
    return model

In [ ]:
model = train_model(10, 60, 6)

In [ ]:
accuracy = compute_accuracy_keras(model, d_list, X_test, Y_test)
print(sum(accuracy.values()) / len(d_list))
for key, acc in accuracy.items():
    print(key, acc)

In [ ]:
res = get_prediction_list_keras(model, d_list)
print(res)
get_creds(res)

In [ ]:
trame_show_result(train_dataset, accuracy, res, 0.8)